<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IVY TRANSPILER

Installing kaggle and uploading the API key necessary to use it.

In [1]:
!pip install -q kaggle
from google.colab import files
from google.colab import userdata
import os
files.upload(); #Upload kaggle.json - you can get from the kaggle account settings, from the API section.

Saving kaggle.json to kaggle.json


Installing packages necessary to use torch's transformers.

In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 5.3 MB/s eta 0:00:00


To use the API, credentials need to be copied into the kaggle folder. If everything works, the output will show the list of available datasets.

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                        title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                        Online Food Dataset                                  3KB  2024-03-02 18:50:30          14075        279  0.9411765        
bhavikjikadara/student-study-performance                   Student Study Performance                            9KB  2024-03-07 06:14:09           7325        110  1.0              
lovishbansal123/sales-of-a-supermarket                     Sales of a Supermarket                              36KB  2024-03-26 12:38:06           1277         27  1.0              
syedanwarafridi/vehicle-sales-data                         Vehicle Sales Data             

Preparing the ivy library.

In [4]:
#Insert the correct user when cloning the repos. Make sure that they are up-to-date.

!git clone "https://github.com/Kacper-W-Kozdon/demos.git"
!git clone "https://github.com/Kacper-W-Kozdon/ivy.git"
!pip install -U -q paddlepaddle ivy accelerate>=0.21.0 mlflow datasets>=2.14.5 nlp 2>/dev/null

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (753/753), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 3152 (delta 660), reused 556 (delta 556), pack-reused 2399
Receiving objects: 100% (3152/3152), 16.80 MiB | 18.21 MiB/s, done.
Resolving deltas: 100% (1820/1820), done.
Cloning into 'ivy'...
remote: Enumerating objects: 189066, done.
remote: Counting objects: 100% (863/863), done.
remote: Compressing objects: 100% (379/379), done.
remote: Total 189066 (delta 600), reused 637 (delta 484), pack-reused 188203
Receiving objects: 100% (189066/189066), 164.31 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (152244/152244), done.


Next: import the ivy library and get the dataset.

In [5]:
import ivy
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

 96% 208M/216M [00:01<00:00, 133MB/s] 
100% 216M/216M [00:01<00:00, 128MB/s]
Archive:  /content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-balanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-unbalanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarc.csv.gz  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv  


Import the libraries suggested in the model which is to be transpiled.

In [6]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays
import paddle

# Libraries to accompany torch's transformers
import tqdm
import boto3
import requests
import regex
import sentencepiece
import sacremoses

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import tensorflow as tf
import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

import accelerate

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

Set the seeds.

In [7]:
tf.keras.utils.set_random_seed(0)
torch.manual_seed(0)
paddle.seed(0)

Get the API key for ivy transpiler from your account and upload it to the project. Move it to the correct directory.

In [8]:
files.upload(); #Upload key.pem - you can get from the kaggle account settings, from the API section.

Saving key.pem to key.pem


In [9]:
!mkdir ~/.ivy #It might be necessary to change ".ivy" to "ivy".
!cp key.pem ~/.ivy/

First we're loading the tokenizer and the model from torch. All of the basic set-up instructions can be found here: https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/huggingface_pytorch-transformers.ipynb#scrollTo=72d8f2de

In [10]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

sequence_classifier = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-cased')

Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *

In [12]:
ivy.set_backend("tensorflow")
ivy.set_soft_device_mode(True)
#tokenizer_tf = ivy.transpile(tokenizer, source="torch", to="tensorflow")
#model_pd = ivy.to_ivy_module(model)
#model_pd = model_pd.trace_graph()
#model_pd = model_pd.set_backend("tensorflow")
model_tf = ivy.transpile(model, source="torch", to="tensorflow")
sequence_classifier_tf = ivy.transpile(sequence_classifier, source="torch", to="tensorflow")

In [ ]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")
df = df.drop_duplicates()
df = df.rename(columns={'comment': 'title'})
df = df[['label', 'title']]
df = df[~df['label'].isnull()]
df = df[~df['title'].isnull()]
df.sample(5)

In [ ]:
def count_words(text: str) -> int:
  return len(text.split())

def count_symbols(text: str) -> int:
  return len("".join(text.split()))

def symbol_to_word_ratio(text: str) -> float:
  return count_symbols(text)/count_words(text)

def upper_lower_ratio(text: str) -> float:
  text = "".join(text.split())
  return sum(1 for c in text if c.isupper())/(max([sum(1 for c in text if c.islower()), 1]))

df['word_count'] = df["title"].apply(count_words)
df['symbol_count'] = df["title"].apply(count_symbols)
df["upper_lower_ratio"] = df["title"].apply(upper_lower_ratio)
df["symbol_to_word_ratio"] = df["title"].apply(symbol_to_word_ratio)
df.sample(5)

A few plots to see some some characteristics of the data.

In [ ]:
df_no_sarc = df.where(df["label"] == 0)
df_no_sarc = df_no_sarc.where(df_no_sarc["word_count"] <= 51)
df_sarc = df.where(df["label"] == 1)
df_sarc = df_sarc.where(df_sarc["word_count"] <= 51)
df_no_sarc = df_no_sarc[np.isfinite(df_no_sarc["word_count"])]
df_sarc = df_sarc[np.isfinite(df_sarc["word_count"])]
plt.style.use('_mpl-gallery-nogrid')

hist_df_no_sarc, bin_edges_no = np.histogram(df_no_sarc["word_count"].values, density=True)
hist_df_sarc, bin_edges = np.histogram(df_sarc["word_count"].values, density=True)
# plot:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

bin_mids_no = [(bin_edges_no[i+1] + bin_edges_no[i])/2 for i in range(len(bin_edges_no) - 1)]
bin_mids = [(bin_edges[i+1] + bin_edges[i])/2 for i in range(len(bin_edges) - 1)]
ax1.bar(bin_mids_no, hist_df_no_sarc, width=bin_edges_no[1] - bin_edges_no[0])
ax2.bar(bin_mids, hist_df_sarc, width=bin_edges[1] - bin_edges[0])
ax1.set_title("Hist no sarcasm")
ax1.set_ylabel("density")
ax1.set_xlabel("word count")
ax1.set_xticks(bin_edges_no)
ax1.grid(True)
ax2.set_title("Hist sarcasm")
ax2.set_xlabel("word count")
ax2.set_xticks(bin_edges)
ax2.grid(True)
plt.show()

In [ ]:
df_no_sarc = df.where(df["label"] == 0)
df_no_sarc = df_no_sarc.where(df_no_sarc["symbol_count"] <= 201)
df_sarc = df.where(df["label"] == 1)
df_sarc = df_sarc.where(df_sarc["symbol_count"] <= 201)
df_no_sarc = df_no_sarc[np.isfinite(df_no_sarc["symbol_count"])]
df_sarc = df_sarc[np.isfinite(df_sarc["symbol_count"])]
plt.style.use('_mpl-gallery-nogrid')

hist_df_no_sarc, bin_edges_no = np.histogram(df_no_sarc["symbol_count"].values, density=True)
hist_df_sarc, bin_edges = np.histogram(df_sarc["symbol_count"].values, density=True)
# plot:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

bin_mids_no = [(bin_edges_no[i+1] + bin_edges_no[i])/2 for i in range(len(bin_edges_no) - 1)]
bin_mids = [(bin_edges[i+1] + bin_edges[i])/2 for i in range(len(bin_edges) - 1)]
ax1.bar(bin_mids_no, hist_df_no_sarc, width=bin_edges_no[1] - bin_edges_no[0])
ax2.bar(bin_mids, hist_df_sarc, width=bin_edges[1] - bin_edges[0])
ax1.set_title("Hist no sarcasm")
ax1.set_ylabel("density")
ax1.set_xlabel("symbol count")
ax1.set_xticks(bin_edges_no)
ax1.grid(True)
ax2.set_title("Hist sarcasm")
ax2.set_xlabel("symbol count")
ax2.set_xticks(bin_edges)
ax2.grid(True)
plt.show()

In [ ]:
df_no_sarc = df.where(df["label"] == 0)
df_no_sarc = df_no_sarc.where(df_no_sarc["upper_lower_ratio"] <= 0.3)
df_sarc = df.where(df["label"] == 1)
df_sarc = df_sarc.where(df_sarc["upper_lower_ratio"] <= 0.3)
df_no_sarc = df_no_sarc[np.isfinite(df_no_sarc["upper_lower_ratio"])]
df_sarc = df_sarc[np.isfinite(df_sarc["upper_lower_ratio"])]
plt.style.use('_mpl-gallery-nogrid')

hist_df_no_sarc, bin_edges_no = np.histogram(df_no_sarc["upper_lower_ratio"].values, density=True)
hist_df_sarc, bin_edges = np.histogram(df_sarc["upper_lower_ratio"].values, density=True)
# plot:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

bin_mids_no = [(bin_edges_no[i+1] + bin_edges_no[i])/2 for i in range(len(bin_edges_no) - 1)]
bin_mids = [(bin_edges[i+1] + bin_edges[i])/2 for i in range(len(bin_edges) - 1)]
ax1.bar(bin_mids_no, hist_df_no_sarc, width=bin_edges_no[1] - bin_edges_no[0])
ax2.bar(bin_mids, hist_df_sarc, width=bin_edges[1] - bin_edges[0])
ax1.set_title("Hist no sarcasm")
ax1.set_ylabel("density")
ax1.set_xlabel("upper/lower ratio")
ax1.set_xticks(bin_edges_no)
ax1.grid(True)
ax2.set_title("Hist sarcasm")
ax2.set_xlabel("upper/lower ratio")
ax2.set_xticks(bin_edges)
ax2.grid(True)
plt.show()

In [ ]:
df_no_sarc = df.where(df["label"] == 0)
df_no_sarc = df_no_sarc.where(df_no_sarc["symbol_to_word_ratio"] <= 11)
df_sarc = df.where(df["label"] == 1)
df_sarc = df_sarc.where(df_sarc["symbol_to_word_ratio"] <= 11)
df_no_sarc = df_no_sarc[np.isfinite(df_no_sarc["symbol_to_word_ratio"])]
df_sarc = df_sarc[np.isfinite(df_sarc["symbol_to_word_ratio"])]
plt.style.use('_mpl-gallery-nogrid')

hist_df_no_sarc, bin_edges_no = np.histogram(df_no_sarc["symbol_to_word_ratio"].values, density=True)
hist_df_sarc, bin_edges = np.histogram(df_sarc["symbol_to_word_ratio"].values, density=True)
# plot:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

bin_mids_no = [(bin_edges_no[i+1] + bin_edges_no[i])/2 for i in range(len(bin_edges_no) - 1)]
bin_mids = [(bin_edges[i+1] + bin_edges[i])/2 for i in range(len(bin_edges) - 1)]
ax1.bar(bin_mids_no, hist_df_no_sarc, width=bin_edges_no[1] - bin_edges_no[0])
ax2.bar(bin_mids, hist_df_sarc, width=bin_edges[1] - bin_edges[0])
ax1.set_title("Hist no sarcasm")
ax1.set_ylabel("density")
ax1.set_xlabel("symbols/words ratio")
ax1.set_xticks(bin_edges_no)
ax1.grid(True)
ax2.set_title("Hist sarcasm")
ax2.set_xlabel("symbols/words ratio")
ax2.set_xticks(bin_edges)
ax2.grid(True)
plt.show()

Checking if the tokenizer, encoder/decoder and classifier work.

In [ ]:
input = df["title"][1]
print(f"The raw input: \n{input}\n")
token = tokenizer(input, return_tensors="pt", add_special_tokens=True)
print(f"The token: \n{token}\n")
with torch.no_grad():
  encoded_token = model(**token)
print(f"The encoded token: \n{encoded_token}\n")

Checking if the transpiled tokenizer, encoder/decoder and classifier work.

In [ ]:
input = df["title"][1]
print(f"The raw input: \n{input}\n")
token = tokenizer.encode(input, return_tensors="tf", add_special_tokens=True)
print(f"The token: \n{token}\n")
#input_ids, token_type_ids, attention_mask = token["input_ids"], token["token_type_ids"], token["attention_mask"]
encoded_token = model_tf(token)
print(f"The encoded token: \n{encoded_token}\n")

A quick check whether transpiling to paddle works as intended.

In [ ]:
class Network(torch.nn.Module):

    def __init__(self):
     super().__init__()
     self._linear = torch.nn.Linear(3, 3)

    def forward(self, x):
     return self._linear(x)

x = torch.tensor([1., 2., 3.])
net = Network()
net(x)

In [ ]:
ivy.set_backend("paddle")
net_pd = ivy.transpile(net, source="torch", to="paddle")
x_pd = paddle.to_tensor([1., 2., 3.])
net_pd(x_pd)

Setting up the classifier based on BERT's sequence classifier model.

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, num_classes=2):
        super(Classifier, self).__init__()
        self.tokenizer = tokenizer
        self.model = sequence_classifier
        self.softmax = torch.nn.functional.softmax
        self.argmax = torch.argmax

    def forward(self, x):
        x = self.tokenizer(x, return_tensors="pt", add_special_tokens=True)
        x = self.model(**x)
        x = self.softmax(x["logits"])
        x = self.argmax(x, dim=1)
        return x

input = df["title"][1]
print(input)
classifier = Classifier()
print(classifier(input))

ivy.set_backend("paddle")
classifier_pd = ivy.transpile(classifier, source="torch", to="paddle")
print(classifier_pd(input))
print(f"Layers: {classifier_pd.sublayers()}")

In [ ]:
print(classifier_pd(input).argmax())
input2 = df["title"][5]
print(classifier_pd(input2).argmax(), classifier_pd(input2))


Setting up the training and training the model.

In [ ]:
def one_hot(input):

  input = paddle.to_tensor(input)
  return paddle.nn.functional.one_hot(input, num_classes=2)

sample = df.sample(5)
sample['label'] = sample['label'].apply(one_hot, "columns")

type(sample["label"])
for label in sample["label"]:
  print(label)


In [ ]:
df.drop(['word_count',	'symbol_count',	'upper_lower_ratio',	'symbol_to_word_ratio'], axis=1, inplace=True)
gc.collect()

In [ ]:
import paddle.distributed as dist
def one_hot(input):
  input = paddle.to_tensor(input)
  return paddle.nn.functional.one_hot(input, num_classes=2)



# if type(df['label'][1]) is np.int64:
#   df['label_one_hot'] = df['label'].apply(one_hot, "columns")

# train_dataset = df[['title', 'label_one_hot']]
# test_dataset = df[['title', 'label_one_hot']]
train_dataset = df[['title', 'label']]
test_dataset = df[['title', 'label']]

In [ ]:
# sample_train = train_dataset.sample(5)
# for entry, label in zip(sample_train["title"], sample_train["label_one_hot"]):
#   print(entry, label)

Switching from pandas.dataframe to paddle.io.Dataset for the convenience of use and compatibility.

In [ ]:
import numpy as np
from paddle.io import Dataset

# define a random dataset
class pd_Dataset(Dataset):
    def __init__(self, df):
      super().__init__()
      self.num_samples = df['title'].size
      self.data = [[entry[0], entry[1]] for entry in zip(df["title"], df["label"])]

    def __getitem__(self, idx):
        title = self.data[idx][0]
        label = self.data[idx][1]
        return title, label

    def __len__(self):
        return self.num_samples

train_dataset_pd = pd_Dataset(train_dataset)

train_loader = paddle.io.DataLoader(train_dataset_pd, batch_size=64, shuffle=True)


In [ ]:

def train(model):

  parameters = model.parameters()
  adam = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
  loss_fn = paddle.nn.CrossEntropyLoss()
  metric = paddle.metric.Accuracy()
  epochs = 2
  classifier = paddle.DataParallel(model)

  for epoch in range(epochs):
    for batch_id, data in enumerate(train_loader()):
      x_data = data[0]
      y_data = data[1]
      predicts = classifier(x_data)
      acc = paddle.metric.accuracy(predicts, y_data)
      loss = loss_fn(predicts, y_data)
      loss.backward()
      if batch_id % 100 == 0:
          print("epoch: {}, batch_id: {}, loss is: {}, acc is: {}".format(epoch, batch_id, loss.numpy(), acc.numpy()))
      adam.step()
      adam.clear_grad()

In [ ]:
for batch_id, data in enumerate(train_loader()):
  print(data[0], data[1])
  if batch_id >= 1:
    break

In [ ]:
train(classifier_pd)